I have built this agent which will work for a retail company which helps user in deciding what to buy. This includes sqlite DB, Tool to call local function, gradio and LLM config.

Configure the local DB :

In [2]:
import sqlite3

DB = "items.db"

with sqlite3.connect(DB) as conn:
    cursor = conn.cursor()
    cursor.execute('DROP TABLE IF EXISTS items')
    cursor.execute('''CREATE TABLE items (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        item TEXT NOT NULL,
        price REAL NOT NULL,
        brand TEXT NOT NULL
    )''')
    conn.commit()


def set_item_details(item, price, brand):
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('INSERT INTO items (item, price, brand) VALUES (?, ?, ?)', (item.lower(), price, brand))
        conn.commit()

In [3]:
item_details = [
    ("shoes", 799, "Nike"),
    ("hat", 399, "Puma"),
    ("bags", 1420, "woodland"),
    ("shorts", 1299, "adidas"),
    ("pants", 1699, "adidas"),
    ("pants", 4699, "PE"),
    ("shorts", 4299, "PE"),
]
for item, price, brand in item_details:
    set_item_details(item, price, brand)

In [ ]:
def get_item_details(item):
    print(f"DATABASE TOOL CALLED: Getting details for {item}", flush=True)
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT id, item, price, brand FROM items WHERE item = ?', (item.lower(),))
        rows = cursor.fetchall()
        if rows:
            lines = [f"ID: {r[0]}, Item: {r[1]}, Price: ${r[2]}, Brand: {r[3]}" for r in rows]
            return "\n".join(lines)
        else:
            return "No details available for this item"

get_item_details("PANTS")

Creating a tool to be used by LLM : 

In [ ]:
item_function = {
    "name": "get_item_details",
    "description": "Get the price brand details of a perticular item user is asking.",
    "parameters": {
        "type": "object",
        "properties": {
            "item": {
                "type": "string",
                "description": "the item whose details are needed",
            },
        },
        "required": ["item"],
        "additionalProperties": False
    }
}
tools = [{"type": "function", "function": item_function}]
tools

Adding Model details:

In [ ]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI

openai_api_key = os.getenv('GOOGLE_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = 'gemini-2.5-flash-lite'
gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
openai = OpenAI(api_key=openai_api_key, base_url=gemini_url)

In [ ]:
system_message = """
You are a helpful assistant for an retail industry 
who only give responses based on the data avaialble in item table which can be called using the tool.
Give short, courteous answers, no more than 1 sentence.
Always be accurate. If you don't know the answer, say so.
"""

In [ ]:
def handle_tool_calls(message):
    responses = []
    print("TOOL IS CALLED HERE")
    for tool_call in message.tool_calls:
        if tool_call.function.name == "get_item_details":
            arguments = json.loads(tool_call.function.arguments)
            item = arguments.get('item')
            print(f"calling hetitemdetails with this item = {item}")
            item_details = get_item_details(item)
            responses.append({
                "role": "tool",
                "content": item_details,
                "tool_call_id": tool_call.id
            })
    return responses

In [ ]:
def chat(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    print(response)
    while response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        responses = handle_tool_calls(message)
        messages.append(message)
        messages.extend(responses)
        response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    
    return response.choices[0].message.content

In [ ]:
import gradio as gr

gr.ChatInterface(fn=chat, type="messages").launch()